# extracting data from s3 storage to python using boto3

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [2]:
data=pd.read_csv("preprocessed_data_model_B.csv")

In [3]:
data.columns

Index(['Unnamed: 0', 'Gender', 'Married', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Loan_Status', 'Applicant_log',
       'CoapplicantIncome_log', 'LoanAmount_log', 'new_income', 'loan', 'EMI',
       'Dependents_1', 'Dependents_2', 'Dependents_3',
       'Property_Area_Semiurban', 'Property_Area_Urban'],
      dtype='object')

In [4]:
dataset=data.drop("Unnamed: 0",axis=1)

In [5]:
len(dataset.columns)

21

In [6]:
dataset.isna().sum()

Gender                     0
Married                    0
Education                  0
Self_Employed              0
ApplicantIncome            0
CoapplicantIncome          0
LoanAmount                 0
Loan_Amount_Term           0
Credit_History             0
Loan_Status                0
Applicant_log              0
CoapplicantIncome_log      0
LoanAmount_log             0
new_income                 0
loan                       0
EMI                        0
Dependents_1               0
Dependents_2               0
Dependents_3               0
Property_Area_Semiurban    0
Property_Area_Urban        0
dtype: int64

# now lets rearrange our columns

In [7]:
loan_status = dataset.pop('Loan_Status')
dataset.insert(len(dataset.columns), 'Loan_Status', loan_status)

In [8]:
#dataset

# now we r going  to split the data to independent and dependent

In [9]:
independent=dataset.iloc[:,:-1]
#independent


In [10]:
dependent=dataset.iloc[:,-1]
#dependent

# now lets split to train and test set

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=.20,random_state=False)

In [13]:
y_train,y_test

(90     1
 533    0
 452    0
 355    1
 266    1
       ..
 277    1
 9      0
 359    1
 192    0
 559    1
 Name: Loan_Status, Length: 491, dtype: int64,
 454    1
 52     0
 536    1
 469    0
 55     1
       ..
 337    1
 376    1
 278    1
 466    0
 303    1
 Name: Loan_Status, Length: 123, dtype: int64)

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled= scaler.fit_transform(x_train)
x_test_scaled= scaler.transform(x_test)

In [15]:
x_train = pd.DataFrame(x_train_scaled, columns=x_train.columns)
x_test = pd.DataFrame(x_test_scaled, columns=x_test.columns)

# now lets do feature selection using XGBoost

In [16]:
#!pip install xgboost

In [17]:
from xgboost import XGBClassifier
from sklearn.feature_selection import RFE

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


In [18]:
xgb=XGBClassifier()
selector= RFE(estimator=xgb,n_features_to_select=10)

In [19]:
selector.fit(x_train,y_train)

RFE(estimator=XGBClassifier(base_score=None, booster=None, callbacks=None,
                            colsample_bylevel=None, colsample_bynode=None,
                            colsample_bytree=None, device=None,
                            early_stopping_rounds=None,
                            enable_categorical=False, eval_metric=None,
                            feature_types=None, feature_weights=None,
                            gamma=None, grow_policy=None, importance_type=None,
                            interaction_constraints=None, learning_rate=None,
                            max_bin=None, max_cat_threshold=None,
                            max_cat_to_onehot=None, max_delta_step=None,
                            max_depth=None, max_leaves=None,
                            min_child_weight=None, missing=nan,
                            monotone_constraints=None, multi_strategy=None,
                            n_estimators=None, n_jobs=None,
                            num_parallel_tree=None, ...),
    n_features_to_select=10)

In [20]:
best_features=x_train.columns[selector.support_]
best_features

Index(['Gender', 'Education', 'Self_Employed', 'ApplicantIncome',
       'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'EMI',
       'Dependents_2', 'Property_Area_Semiurban'],
      dtype='object')

In [21]:
print(f"best features are:  {best_features}")

best features are:  Index(['Gender', 'Education', 'Self_Employed', 'ApplicantIncome',
       'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'EMI',
       'Dependents_2', 'Property_Area_Semiurban'],
      dtype='object')


In [22]:
best_features=list(best_features)

In [23]:
x_train=x_train[best_features]

In [24]:
x_test=x_test[best_features]

In [25]:
# now lets create models and train them

In [26]:
#!pip install xgboost

In [27]:
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

In [28]:
models={"Random Forest": RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42),
    "XGBoost": XGBClassifier(n_estimators=100, learning_rate=0.1, use_label_encoder=False, eval_metric='logloss', random_state=42),
    "Logistic Regression": LogisticRegression(solver='liblinear', random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "Naive Bayes": GaussianNB()}

In [29]:
accuracy={}
for name, model in models.items():
    model.fit(x_train,y_train)
    y_predict=model.predict(x_test)
    acc=accuracy_score(y_predict,y_test)
    accuracy[name]=acc

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [01:20:29] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [30]:
accuracy

{'Random Forest': 0.6829268292682927,
 'XGBoost': 0.6666666666666666,
 'Logistic Regression': 0.7235772357723578,
 'KNN': 0.6341463414634146,
 'Naive Bayes': 0.7479674796747967}

In [31]:
from sklearn.model_selection import GridSearchCV

In [33]:
gnb = GaussianNB()
param_grid = {'var_smoothing': [1e-09, 1e-08, 1e-07, 1e-06, 1e-05]}
grid_search = GridSearchCV(estimator=gnb, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=GaussianNB(),
             param_grid={'var_smoothing': [1e-09, 1e-08, 1e-07, 1e-06, 1e-05]},
             scoring='accuracy')

In [34]:
print(grid_search.best_params_)

{'var_smoothing': 1e-09}


In [35]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_test)
final_acc = accuracy_score(y_test, y_pred)
final_acc

0.7479674796747967

In [36]:
import joblib
import os

In [37]:
os.makedirs("models",exist_ok=True)
joblib.dump(best_model,"models/modelB.pkl")

['models/modelB.pkl']